# Confident | 自信

> [DeepEval](https://confident-ai.com)パッケージはLLMのユニットテスト用です。Confidentを使用することで、ユニットテストと統合テストを組み合わせることにより、迅速なイテレーションを通じて誰もが堅牢な言語モデルを構築できます。私たちは合成データの作成からテストに至るまでのイテレーションの各ステップでサポートを提供しています。
>
> > [DeepEval](https://confident-ai.com) package for unit testing LLMs.
> > Using Confident, everyone can build robust language models through faster iterations
> > using both unit testing and integration testing. We provide support for each step in the iteration
> > from synthetic data creation to testing.




このガイドでは、LLMsのパフォーマンスをテストして測定する方法をご紹介します。私たちのコールバックを使用してパフォーマンスを測定する方法と、ご自身でメトリックを定義し、それを私たちのダッシュボードに記録する方法を示します。

> In this guide we will demonstrate how to test and measure LLMs in performance. We show how you can use our callback to measure performance and how you can define your own metric and log them into our dashboard.

DeepEvalは以下の機能も提供します：

> DeepEval also offers:

* 合成データの生成方法

  > How to generate synthetic data

* パフォーマンスの測定方法

  > How to measure performance

* 時間をかけて結果を監視し、レビューするダッシュボード

  > A dashboard to monitor and review results over time





## Installation and Setup | インストールとセットアップ




In [ ]:
!pip install deepeval --upgrade

### Getting API Credentials | APIクレデンシャルの取得

DeepEval APIの認証情報を取得するには、次の手順に従ってください：

> To get the DeepEval API credentials, follow the next steps:

1. https://app.confident-ai.com にアクセスしてください。

   > Go to https://app.confident-ai.com

2. 「Organization」をクリックしてください

   > Click on "Organization"

3. APIキーをコピーしてください。

   > Copy the API Key.


ログインする際には、`implementation`（実装）の名前を設定するよう求められます。実装名は、実装のタイプを記述するために必要です。（プロジェクトにどのような名前を付けたいか考えてみてください。説明的であることをお勧めします。）

> When you log in, you will also be asked to set the `implementation` name. The implementation name is required to describe the type of implementation. (Think of what you want to call your project. We recommend making it descriptive.)




In [11]:
!deepeval login

### Setup DeepEval | DeepEvalのセットアップ

`DeepEvalCallbackHandler`をデフォルトで使用して、追跡したいメトリクスを設定することができます。ただし、現時点でサポートされているメトリクスは限られています（さらに多くのメトリクスが間もなく追加される予定です）。現在サポートしているのは以下の通りです：

> You can, by default, use the `DeepEvalCallbackHandler` to set up the metrics you want to track. However, this has limited support for metrics at the moment (more to be added soon). It currently supports:

* [回答の関連性](https://docs.confident-ai.com/docs/measuring_llm_performance/answer_relevancy)

  > [Answer Relevancy](https://docs.confident-ai.com/docs/measuring_llm_performance/answer_relevancy)

* [バイアス](https://docs.confident-ai.com/docs/measuring_llm_performance/debias)

  > [Bias](https://docs.confident-ai.com/docs/measuring_llm_performance/debias)

* [Toxicness](https://docs.confident-ai.com/docs/measuring_llm_performance/non_toxic)

  > [Toxicness](https://docs.confident-ai.com/docs/measuring_llm_performance/non_toxic)





In [ ]:
from deepeval.metrics.answer_relevancy import AnswerRelevancy

# Here we want to make sure the answer is minimally relevant
answer_relevancy_metric = AnswerRelevancy(minimum_score=0.5)

## Get Started | はじめに




`DeepEvalCallbackHandler`を使用するには、`implementation_name`が必要です。

> To use the `DeepEvalCallbackHandler`, we need the `implementation_name`.




In [ ]:
from langchain.callbacks.confident_callback import DeepEvalCallbackHandler

deepeval_callback = DeepEvalCallbackHandler(
    implementation_name="langchainQuickstart", metrics=[answer_relevancy_metric]
)

### Scenario 1: Feeding into LLM | シナリオ1: LLMへの入力

その後、それをOpenAIのLLMにフィードすることができます。

> You can then feed it into your LLM with OpenAI.




In [7]:
from langchain.llms import OpenAI

llm = OpenAI(
    temperature=0,
    callbacks=[deepeval_callback],
    verbose=True,
    openai_api_key="<YOUR_API_KEY>",
)
output = llm.generate(
    [
        "What is the best evaluation tool out there? (no bias at all)",
    ]
)

LLMResult(generations=[[Generation(text='\n\nQ: What did the fish say when he hit the wall? \nA: Dam.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe Moon \n\nThe moon is high in the midnight sky,\nSparkling like a star above.\nThe night so peaceful, so serene,\nFilling up the air with love.\n\nEver changing and renewing,\nA never-ending light of grace.\nThe moon remains a constant view,\nA reminder of life’s gentle pace.\n\nThrough time and space it guides us on,\nA never-fading beacon of hope.\nThe moon shines down on us all,\nAs it continues to rise and elope.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nQ. What did one magnet say to the other magnet?\nA. "I find you very attractive!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text="\n\nThe world is charged with the grandeur of God.\nIt will flame out, like shining from shook foil;\nIt gathers to a greatness, like t

その後、`is_successful()` メソッドを呼び出して、メトリックが成功したかどうかを確認することができます。

> You can then check the metric if it was successful by calling the `is_successful()` method.




In [ ]:
answer_relevancy_metric.is_successful()
# returns True/False

それを実行したら、私たちのダッシュボードが下に表示されるはずです。

> Once you have ran that, you should be able to see our dashboard below.

![Dashboard](https://docs.confident-ai.com/assets/images/dashboard-screenshot-b02db73008213a211b1158ff052d969e.png)




### Scenario 2: Tracking an LLM in a chain without callbacks | シナリオ2: コールバックなしでチェーン内のLLMを追跡する

コールバックを使用せずにチェーン内のLLMを追跡するには、末尾でそれにプラグインすることができます。

> To track an LLM in a chain without callbacks, you can plug into it at the end.

以下に示すように、シンプルなチェーンを定義することから始めましょう。

> We can start by defining a simple chain as shown below.




In [ ]:
import requests
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

text_file_url = "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"

openai_api_key = "sk-XXX"

with open("state_of_the_union.txt", "w") as f:
    response = requests.get(text_file_url)
    f.write(response.text)

loader = TextLoader("state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=openai_api_key),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
)

# Providing a new question-answering pipeline
query = "Who is the president?"
result = qa.run(query)

チェーンを定義した後、手動で回答の類似性を確認することができます。

> After defining a chain, you can then manually check for answer similarity.




In [ ]:
answer_relevancy_metric.measure(result, query)
answer_relevancy_metric.is_successful()

### What's next? | 次は何ですか？

独自のカスタムメトリクスを[こちら](https://docs.confident-ai.com/docs/quickstart/custom-metrics)で作成できます。

> You can create your own custom metrics [here](https://docs.confident-ai.com/docs/quickstart/custom-metrics).

DeepEvalには、[自動的にユニットテストを作成する](https://docs.confident-ai.com/docs/quickstart/synthetic-data-creation)機能や、[事実の整合性をテストする](https://docs.confident-ai.com/docs/measuring_llm_performance/factual_consistency)機能など、他にもさまざまな特徴があります。

> DeepEval also offers other features such as being able to [automatically create unit tests](https://docs.confident-ai.com/docs/quickstart/synthetic-data-creation), [tests for hallucination](https://docs.confident-ai.com/docs/measuring_llm_performance/factual_consistency).

興味がある方は、こちらのGithubリポジトリ <https://github.com/confident-ai/deepeval> をチェックしてください。LLMの性能向上に関するPRや議論をいつでも歓迎します。

> If you are interested, check out our Github repository here <https://github.com/confident-ai/deepeval>. We welcome any PRs and discussions on how to improve LLM performance.


